# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [60]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

path_list = []

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    path_list.append(file_path_list)
    #print(file_path_list)
path_list = path_list[0]
path_list




/home/workspace


['/home/workspace/event_data/2018-11-18-events.csv',
 '/home/workspace/event_data/2018-11-15-events.csv',
 '/home/workspace/event_data/2018-11-16-events.csv',
 '/home/workspace/event_data/2018-11-09-events.csv',
 '/home/workspace/event_data/2018-11-11-events.csv',
 '/home/workspace/event_data/2018-11-17-events.csv',
 '/home/workspace/event_data/2018-11-14-events.csv',
 '/home/workspace/event_data/2018-11-02-events.csv',
 '/home/workspace/event_data/2018-11-27-events.csv',
 '/home/workspace/event_data/2018-11-13-events.csv',
 '/home/workspace/event_data/2018-11-26-events.csv',
 '/home/workspace/event_data/2018-11-03-events.csv',
 '/home/workspace/event_data/2018-11-19-events.csv',
 '/home/workspace/event_data/2018-11-25-events.csv',
 '/home/workspace/event_data/2018-11-04-events.csv',
 '/home/workspace/event_data/2018-11-29-events.csv',
 '/home/workspace/event_data/2018-11-01-events.csv',
 '/home/workspace/event_data/2018-11-06-events.csv',
 '/home/workspace/event_data/2018-11-24-events

In [61]:
for file in path_list:
    print(file)

/home/workspace/event_data/2018-11-18-events.csv
/home/workspace/event_data/2018-11-15-events.csv
/home/workspace/event_data/2018-11-16-events.csv
/home/workspace/event_data/2018-11-09-events.csv
/home/workspace/event_data/2018-11-11-events.csv
/home/workspace/event_data/2018-11-17-events.csv
/home/workspace/event_data/2018-11-14-events.csv
/home/workspace/event_data/2018-11-02-events.csv
/home/workspace/event_data/2018-11-27-events.csv
/home/workspace/event_data/2018-11-13-events.csv
/home/workspace/event_data/2018-11-26-events.csv
/home/workspace/event_data/2018-11-03-events.csv
/home/workspace/event_data/2018-11-19-events.csv
/home/workspace/event_data/2018-11-25-events.csv
/home/workspace/event_data/2018-11-04-events.csv
/home/workspace/event_data/2018-11-29-events.csv
/home/workspace/event_data/2018-11-01-events.csv
/home/workspace/event_data/2018-11-06-events.csv
/home/workspace/event_data/2018-11-24-events.csv
/home/workspace/event_data/2018-11-22-events.csv
/home/workspace/even

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [63]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [64]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [65]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [66]:
# TO-DO: Create a Keyspace (Sparkify)

create_keyspace = """CREATE KEYSPACE IF NOT EXISTS Sparkify
                             WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""

try:
    session.execute(create_keyspace)
except Exception as e:
    print(e)

#### Set Keyspace

In [67]:
# TO-DO: Set KEYSPACE to the keyspace specified above
set_keyspace = "USE Sparkify"

try:
    session.execute(set_keyspace)
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [116]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS album_library "
query = query + "(sessionId text, itemInSession int, song text, artist text, length float, PRIMARY KEY (sessionId, artist, song))"
try:
    session.execute(query)
except Exception as e:
    print(e)



In [123]:
create_statement =  """CREATE TABLE IF NOT EXISTS song_data_by_session (sessionId INT,
        itemInSession INT,
        artist TEXT,
        song TEXT,
        length DOUBLE,
        PRIMARY KEY ((sessionId, itemInSession)))"""
session.execute(create_statement)



In [125]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## TO-DO: Assign the INSERT statements into the `query` variable
        query = """INSERT INTO song_data_by_session (sessionId, itemInSession, artist, song, length) """
        query = query + "VALUES  (%s, %s, %s, %s, %s)"
              
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.        
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`

        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [127]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
select_statement = "select * from song_data_by_session where sessionId= %s and itemInSession= %s "

varify_data = session.execute(select_statement, (338, 4))

for row in varify_data:
    print (row)

Row(sessionid=338, iteminsession=4, artist='Faithless', length=495.3073, song='Music Matters (Mark Knight Dub)')


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [163]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) 
## and user (first and last name)\
## for userid = 10, sessionid = 182

# Create new table song by song by userid and session id
create_statement2 =  """CREATE TABLE IF NOT EXISTS song_by_userid_and_sessionid (userId INT, 
        sessionId INT,
        itemInSession INT,
        artist TEXT,
        song TEXT,
        firstName TEXT,
        lastName TEXT,
        PRIMARY KEY ((userId, sessionId), itemInSession))"""

session.execute(create_statement2)



In [164]:

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = """INSERT INTO song_by_userid_and_sessionid (userId, sessionId, itemInSession, artist, song,
                   firstName, lastName ) """
        query = query + " VALUES  (%s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
            



In [166]:
### select statement for Query 2 above
select_statement2 = "select * from song_by_userid_and_sessionid where userId= %s and sessionId= %s "

varify_data2 = session.execute(select_statement2, (10, 182))

for row in varify_data2:
    print (row)

Row(userid=10, sessionid=182, iteminsession=0, artist='Down To The Bone', firstname='Sylvie', lastname='Cruz', song="Keep On Keepin' On")
Row(userid=10, sessionid=182, iteminsession=1, artist='Three Drives', firstname='Sylvie', lastname='Cruz', song='Greece 2000')
Row(userid=10, sessionid=182, iteminsession=2, artist='Sebastien Tellier', firstname='Sylvie', lastname='Cruz', song='Kilometer')
Row(userid=10, sessionid=182, iteminsession=3, artist='Lonnie Gordon', firstname='Sylvie', lastname='Cruz', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)')


In [168]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history 
## who listened to the song 'All Hands Against His Own'

create_statement3 =  """CREATE TABLE IF NOT EXISTS song_by_username (song TEXT,                
        firstName TEXT,
        lastName TEXT,
        PRIMARY KEY (song, firstName, lastName))"""

session.execute(create_statement3)


                    

In [169]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query3 = """INSERT INTO song_by_username (song,
                   firstName, lastName ) """
        query3 = query3 + " VALUES  (%s, %s, %s)"
        
        session.execute(query3, (line[9], line[1], line[4]))

In [172]:
### select statement for Query 3 above
select_statement3 = "select * from song_by_username where song = %s "

varify_data3 = session.execute(select_statement3, ( 'All Hands Against His Own',))

for row in varify_data3:
    print (row)

Row(song='All Hands Against His Own', firstname='Jacqueline', lastname='Lynch')
Row(song='All Hands Against His Own', firstname='Sara', lastname='Johnson')
Row(song='All Hands Against His Own', firstname='Tegan', lastname='Levine')


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [173]:
query = "drop table song_by_username"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


In [174]:
query = "drop table song_by_userid_and_sessionid"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [175]:
query = "drop table song_data_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [176]:
session.shutdown()
cluster.shutdown()